In [1]:
import numpy as np
import pandas as pd
from scipy.stats import f_oneway
from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
def generateCandidates(Ti,l):
    Wil = [np.array(Ti[i : i + l]) for i in range(0, len(Ti)-l+1, 1)]
    return Wil

In [3]:
def create_Wl(T,l):
    return [generateCandidates(Ti,l) for Ti in T]

In [4]:
def findDistances(S,Wl):
    Ds = []
    for Wil in Wl:
        Ds.append(cdist(np.array([S]),np.vstack(Wil),metric='seuclidean').min())
    return np.array(Ds)

In [5]:
def assessCandidate(S,Ds,C):
    class_groups = []
    for c in np.unique(C):
        class_groups.append(Ds[C==c].tolist())

#     plt.figure(figsize=(10,5))
#     colors = np.array(['red','blue','green'])
#     for c in np.unique(C):
#         sns.distplot(x=Ds[C==c],color= colors[c])
    
    return f_oneway(*class_groups).statistic

In [6]:
def sortByQuality(shapelets):
    return sorted(shapelets, key=lambda tup: tup[1],reverse=True)

In [7]:
def removeSelfSimilar(shapelets):
    queue = shapelets[:]
    df = pd.DataFrame(shapelets)

    keep_df = pd.DataFrame()

    while len(df) != 0:
        pop_item = queue.pop(0)
        s,f,interval = pop_item

        keep_df = pd.concat((keep_df,pd.DataFrame([pop_item])))

        df = df[~df[2].apply(lambda x: interval.overlaps(x))]
        queue = df.values.tolist()
    
    return keep_df.drop(2,axis=1).values.tolist()

In [8]:
def merge(k,kShapelets,shapelets):
    total_shapelets = kShapelets + shapelets
    return sortByQuality(total_shapelets)[:k]

In [9]:
def ShapeletCachedSelection(data,lmin,lmax,k,label):
   # 原始data顺序 样本数 维度数 序列长度
    data = np.transpose(data,axes=(1,0,2))
    # 现在data顺序 维度数 样本数 序列长度
    dim_nums = data.shape[0]
    sample_nums = data.shape[1]
    for dn in tqdm(range(dim_nums)):
        T = data[dn]
        # print(keyPoints)
        precompute_Wl = {l : create_Wl(T,l) for l in np.arange(lmin,lmax+1)}

        kShapelets = []
        for i,Ti in enumerate(T):
            shapelets = []
            for l in np.arange(lmin,lmax+1):
                # S为所有的长度
                Al = precompute_Wl[l]
                Wl,Pl = Al[0],Al[1]
                for index,S in enumerate(Wl[i]):
                     # 和所有样本上的候选者比较
                    #  S 区间，keypoint，松弛长度
                    # Ds,Indx = findDistances(S,Wl)

                    Ds = findDistances(S,Wl)
                    quality = assessCandidate(Ds,label)
                    #  区间值 度量  区间 关键点
                    shapelets.append((S,quality,pd.Interval(Pl[index][1],Pl[index][2]-1,closed='both'),Pl[index][0]))
            shapelets = sortByQuality(shapelets)
            shapelets = removeSelfSimilar(shapelets)
            kShapelets = merge(k,kShapelets,shapelets)
        print(kShapelets)

    # return kShapelets

In [10]:
def load_raw_ts(path, dataset):
    path = path + "raw//" + dataset + "//"
    # 训练集
    x_train = np.load(path + 'X_train.npy')
    x_train = np.transpose(x_train, axes=(0, 2, 1))
    x_test = np.load(path + 'X_test.npy')
    x_test = np.transpose(x_test, axes=(0, 2, 1))
    y_train = np.load(path + 'y_train.npy')
    y_test = np.load(path + 'y_test.npy')
    labels = np.concatenate((y_train, y_test), axis=0)
    nclass = int(np.amax(labels)) + 1
    return x_train, x_test, y_train.reshape(-1), y_test.reshape(-1), nclass

In [13]:
data_train, data_test, target_train, target_test, nclass = load_raw_ts("D://tmppro//data//", "StandWalkJump")
sample_nums = data_train.shape[0]
dim_nums = data_train.shape[1]
series_length = data_train.shape[2]

import random

# # 这里的N指的是序列的样本数量
# # N = len(T)
# # N_test = len(T_test)
# # print(N,N_test)
# # N=200
# #N_test = 5000
lmin = 5
lmax = 10
# 选取shapelet的数量
k = 50
alp = random.choice([0.3, 0.4])
blt = random.choice([100, 90, 80, 70, 60, 50])
ylt = random.choice([5,6,7,8])
#
# # min_seq_len = 20
# # max_seq_len = 30
# # T,C = generate_dataset(N,min_seq_len,max_seq_len)
# # T_test,C_test = generate_dataset(N_test,min_seq_len,max_seq_len)

In [16]:
kshapelets = ShapeletCachedSelection(data_train,lmin,lmax,k,target_train)

  0%|          | 0/4 [00:00<?, ?it/s]


ValueError: XA must be a 2-dimensional array.